In [10]:
#Import from the Keras library
from keras import models
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,  MaxPooling2D
from keras import optimizers 
from keras import utils
from keras.models import load_model
from secret import credentials

#Other import statements 
import h5py
import random
import numpy as np
import pandas as pd
import pathlib
import cv2
import pymysql
import os

In [6]:
#The values by which to resize images
nrows = 150
ncols = 150 
channels = 3

In [7]:
def connect(): 
    db_host ='nicu-2019-03-05.c2lckhwrw1as.us-east-1.rds.amazonaws.com'
    db_port = 3306
    db_name = 'nicu'
    db_username = 'jonlee'
    db_password = 'nicu_jon'
    
    '''
    db_host = credentials['db_host'];
    db_port = credentials['db_port'];
    db_name = credentials['db_name'];
    db_username = credentials['db_username']
    db_password = credentials['db_password']
    '''
    
    conn = pymysql.connect(db_host, user=db_username, port=db_port, passwd=db_password, db=db_name)
    return conn

In [62]:
def import_data(cache_path, conn=connect()): 

    curs = conn.cursor()
    
    recording_ids = {}
    xy = {}    
        
    image_query = "SELECT r.id, r.recording_id, r.isCSGM FROM nicu.Video_Raw AS r JOIN nicu.Video_Generated AS g ON r.id=g.raw_id  WHERE (r.recording_id>1) AND (g.RGB_Optical_Flow IS NOT NULL) LIMIT 10"
    try:
        curs.execute(image_query) #(list(recording_ids.keys())))
        for row in curs.fetchall():
            raw_id = row[0]
            rec_id = row[1]
            csgm = row[2]
            if rec_id in recording_ids:
                recording_ids.get(rec_id).append(raw_id)
            else:
                recording_ids.update({rec_id:[raw_id]})
            xy.update({raw_id:[csgm]})
            
    except Exception as e:
        print("Error retrieving ID's", e)
        raise e
        
    for rec_id in recording_ids:
        #cache_path = cache_path+"recording_"+("{:02d}".format(rec_id))
        cache_path = cache_path+'testing'
        if not os.path.exists(cache_path):
            os.mkdir(cache_path)
        raw_id_list = recording_ids.get(rec_id)
        for raw_id in raw_id_list:
            current_input = cache_path+'/'+str(raw_id)+".oflow.png"
            if not os.path.exists(current_input):
                try:
                    image_query = "SELECT RGB_Optical_Flow from Video_Generated WHERE (raw_id=%s)"
                    curs.execute(image_query, (str(raw_id)))
                    for row in curs.fetchall():
                        db_img = row[0]
                        if db_img is not None:
                            img=cv2.imdecode(np.asarray(bytearray(db_img),dtype=np.uint8),cv2.IMREAD_UNCHANGED)
                            cv2.imwrite(current_input,img)
                except Exception as e:
                        print("Error retrieving Optical Flow frame",e)
                        raise e
            xy.get(raw_id).insert(0,cv2.imread(current_input))
    curs.close()
    
    return recording_ids, xy
    


In [52]:
def create_array(raw_ids, xy):    
    image_list=[]
    csgm_list=[]
    
    random.shuffle(raw_ids)
    
    for i in raw_ids:
        if xy.get(i)[0]!=None:
            image_list.append(xy.get(i)[0])
            csgm_list.append(xy.get(i)[1])
    x = np.array(image_list)
    y = np.array(csgm_list)
    return x, y
        

In [6]:
def create_model():    
    model = models.Sequential() 

    #The model will learn 32 filters in this layer
    model.add(Conv2D(32, (3,3), activation = 'relu', input_shape=(150,150,3))) # this applies 32 convolution filters of size 3x3 each
    model.add(Conv2D(32, (3,3), activation='relu'))

    #Max Pooling takes a 2x2 grid and takes the highest value of the grid and resizes based on that max value
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    #Softmax activation is the traditional form of activation for mutliclass classification
    model.add(Dense(1, activation='softmax')) 
    
    model.compile(loss='categorical_hinge', 
              optimizer=optimizers.SGD(lr=1e-4), #SGD = Stochastic Gradient Descent (Can be changed)
              metrics=['acc']) 

    return model


In [7]:
def confusion_matrix(size, exp_values, predicted_values, labels):
    """
    This creates a confusion matrix with the predicted accuracy of the model.
    The ouput is a pandas dataframe with the labels and columns of the different labels inputted. 
    """
    predicted_values = convert_predictions(predicted_values)
    
    #Creates a DataFrame of zeros
    matrix = pd.DataFrame(np.zeros((size,size)) , labels, labels)
    
    #Caculates whether the score was right or wrong 
    for i in range(len(exp_values)):
        for j in range(size):
            if exp_values[i][j] == predicted_values[i][j] and predicted_values[i][j] == 1:
                matrix.iloc[[find_one(predicted_values[i])],[find_one(exp_values[i])]] += 1
            elif exp_values[i][j] == 0 and predicted_values[i][j] == 1:
                matrix.iloc[[find_one(predicted_values[i])],[find_one(exp_values[i])]] += 1
   
    #Calculate diagonal sum and the accuracy of the model
    diagonal_sum = 0
    for i in range(size):
        diagonal_sum += matrix.iloc[i][i]
    
    score = diagonal_sum/len(exp_values)
    
  
    return  matrix, score
    
    
def find_one(l):
    """
    Finds the first occurence of one in a list and returns it.
    """
    for i in range(len(l)): 
        if l[i] == 1: return i
    return 0
            
def convert_predictions(predictions): 
    """
    Converts predictions outputted by a keras model into a list with 1 represented the predicted output and zero 
    in other classes. 
    """
    for p in predictions: 
        max_val = max(p)
        for n in range(len(p)): 
            if p[n] == max_val:
                p[n] = 1
            else:
                p[n] = 0
    return predictions

In [8]:
def runTest():
    matrices = {}
    scores = {}
    model_scores = {}
    
    recording_ids_dict, xy = import_data('NICU_data/')
    
    for i in recording_ids_dict:
        train_ids= []
        test_ids = []
        
        for j in recording_ids_dict:
            if j == i:
                test_ids = recording_ids_dict[j]
            else: 
                train_ids = train_ids.append(recording_ids_dict[j])
        
        x_train, y_train = create_array(train_ids, xy)
        x_test, y_test = create_array(test_ids, xy)
        
        model = create_model()
        
        #Fit the model
        model.fit(x_train, y_train, batch_size = 32, epochs = 5)
        
        #Create predictions and evaluate to find loss and accuaracy
        predict = model.predict(x_test)
        model_score = model.evaluate(x_test, y_test)
        
        matrix,score = confusion_matrix(len(y_test[0]), y_test, predict, labels)
        
        matrices.update({i : matrix})
        print(matrices)
        scores.update({i: score})
        print(scores)
        model_scores.update({i:model_score})
        
    return matrices, scores, model_scores 

In [10]:
labels = ["LeftArm", "RightArm", "LeftLeg","RightLeg"]
names = ["/Isaiah", "/Kaylee", "/Patterson", "/Ryan"]
matrices, scores, model_scores = runTest(names,labels)

with open('LargeHingeResults.txt', 'w') as f:
    for key in matrices:
        f.write("%s\n" % key)
        f.write("%s\n" % matrices[key])
        f.write("%s\n" % scores[key])
        f.write("%s\n" % model_scores[key])

['/Kaylee', '/Patterson', '/Ryan'] are the training names
/Isaiah is being tested on.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/5
2039/2039 [==============================] - 209s 102ms/step - loss: 0.9849 - acc: 0.4836
Epoch 2/5
2039/2039 [==============================] - 745s 365ms/step - loss: 0.4436 - acc: 0.7989
Epoch 3/5
2039/2039 [==============================] - 1188s 583ms/step - loss: 0.2830 - acc: 0.8872
Epoch 4/5
1504/2039 [=====================>........] - ETA: 49s - loss: 0.2363 - acc: 0.9122

KeyboardInterrupt: 